In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, walk, makedirs
from os.path import join, basename, isdir
sys.path.append(str(Path(getcwd()).parent.parent))

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import codecs
import numpy as np
from glob import glob
from epiweeks import Week

from matplotlib import pyplot as plt

In [ ]:
from bulletin import default_input, default_output, tables_path, root

from bulletin.systems import Notifica, CasosConfirmados, eSUS, Sivep
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import normalize_text, normalize_hash, date_hash

from datetime import datetime, date, timedelta

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
default_output = join(default_output,'vacinacao',f'obitos')
    
if not isdir(default_output): 
    makedirs(default_output)
    
default_output

In [ ]:
join_columns_levels = lambda df: df.set_axis([" ".join(map(str,columns)) for columns in df.columns], axis=1)

In [ ]:
situacao_contagio_labels = { -1: 'Não Vacinado / Esquema Primário Incompleto', 0:'Não Vacinado', 1:'Esquema Primário Incompleto', 2:'Esquema Primário Completo ', 3: 'Esquema Primário Completo + Reforço'}
fator_risco_labels = { 0:'Sem Fator de Risco', 1:'Com Fator de Risco' }
mes_labels = {key+1:value for key,value in enumerate(['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'])}
evolucao_labels = {1: 'Recuperados', 2: 'Óbitos', 3: 'Ativos'}
raca_cor_labels = CasosConfirmados().tables['raca_cor'].set_index('id')['raca_cor'].to_dict()

In [ ]:
cc_vacinados = CasosConfirmados('cc_vacinados')
cc_vacinados.load()
casos = cc_vacinados.df

In [ ]:
cc_vacinados.df.columns.tolist()

In [ ]:
casos.loc[casos['raca_cor'].isin([0,6]),'raca_cor'] = 99

In [ ]:
casos = casos.loc[casos['uf_residencia']=='PR'].copy()

casos['fx'] = '60+'
casos.loc[(casos['idade']>=12)&(casos['idade']<60),'fx'] = '12-59'
casos.loc[(casos['idade']<12),'fx'] = '0-11'

In [ ]:
casos.groupby(['data_diagnostico','situacao_contagio'])[['id']].count().unstack('situacao_contagio').droplevel(axis=1,level=0).rename(columns=situacao_contagio_labels,level='situacao_contagio').rolling(7).mean().plot(figsize=(20,10))


In [ ]:
obitos = casos.loc[(casos['evolucao']==2)].copy()
obitos['ano'] = obitos['data_cura_obito'].dt.year
obitos['mes'] = obitos['data_cura_obito'].dt.month

In [ ]:

obitos['situacao_contagio'] = 0
obitos.groupby('situacao_contagio')[['paciente']].count()

obitos.loc[
    (
        (obitos['doses_aplicadas']=='1ª Dose') 
        &
        (obitos['data_cura_obito'] >= obitos['data_aplicacao_1a_dose'])
    ) 
    |
    (
        (obitos['doses_aplicadas']=='1ª Dose,2ª Dose') 
        &
        (obitos['data_cura_obito'] >= obitos['data_aplicacao_1a_dose'])
        &
        (obitos['data_cura_obito'] <= obitos['data_imunizacao']  )
    ) 
    | 
    (
        (obitos['doses_aplicadas']=='Única') 
        & 
        (obitos['data_cura_obito'] >= obitos['data_aplicacao_dose_unica'])
        &
        (obitos['data_cura_obito'] <= obitos['data_imunizacao']  )
    )
, 'situacao_contagio'] = 1

obitos.groupby('situacao_contagio',dropna=False)[['paciente']].count()

obitos.loc[
    obitos['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose',
            '1ª Dose,2ª Dose,Adicional',
            'Única',
            'Única,Adicional',
        ]
    ) 
    &
    (obitos['data_cura_obito'] > obitos['data_imunizacao']  )
    
, 'situacao_contagio'] = 2

obitos.groupby('situacao_contagio',dropna=False)[['paciente']].count()

obitos.loc[
    obitos['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose,Reforço',
            'Única,Reforço',
        ]
    ) 
    &
    (obitos['data_cura_obito'] > obitos['data_imunizacao'])
    &
    (obitos['data_cura_obito'] <= obitos['data_imunizacao_reforco'])

, 'situacao_contagio'] = 2

obitos.groupby('situacao_contagio',dropna=False)[['paciente']].count()

obitos.loc[
    obitos['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose,Reforço',
            'Única,Reforço',
        ]
    ) 
    &
    (obitos['data_cura_obito'] > obitos['data_imunizacao_reforco'])

, 'situacao_contagio'] = 3

obitos_situacao_contagio = obitos.groupby('situacao_contagio',dropna=False)[['paciente']].count().rename(index=situacao_contagio_labels, level='situacao_contagio').rename(columns={'paciente':'quantidade'})
obitos_situacao_contagio.to_csv(join(default_output,'obitos_situacao_contagio.csv'))
obitos_situacao_contagio

In [ ]:
obitos.groupby(['data_cura_obito','situacao_contagio'])[['id']].count().unstack('situacao_contagio').droplevel(axis=1,level=0).rename(columns=situacao_contagio_labels,level='situacao_contagio').rolling(7).mean().plot(figsize=(20,10))

In [ ]:
obitos_situacao_contagio_fator_risco = obitos.groupby(['situacao_contagio','fator_risco'])[['id']].count().unstack('fator_risco').fillna(0).astype(int).droplevel(0,1)
obitos_situacao_contagio_fator_risco = obitos_situacao_contagio_fator_risco.rename(index=situacao_contagio_labels, level='situacao_contagio').rename(columns=fator_risco_labels, level='fator_risco')
obitos_situacao_contagio_fator_risco.to_csv(join(default_output,'obitos_situacao_contagio_fator_risco.csv'))
obitos_situacao_contagio_fator_risco

In [ ]:
obitos_situacao_contagio_faixa_etaria = obitos.groupby(['situacao_contagio','fx'])[['id']].count().unstack('fx').fillna(0).astype(int).droplevel(0,1)
obitos_situacao_contagio_faixa_etaria = obitos_situacao_contagio_faixa_etaria.rename(index=situacao_contagio_labels, level='situacao_contagio')
obitos_situacao_contagio_faixa_etaria.to_csv(join(default_output,'obitos_situacao_contagio_faixa_etaria.csv'))
obitos_situacao_contagio_faixa_etaria

In [ ]:
obitos_situacao_contagio_sexo = obitos.groupby(['situacao_contagio','sexo'])[['id']].count().unstack('sexo').fillna(0).astype(int).droplevel(0,1)
obitos_situacao_contagio_sexo = obitos_situacao_contagio_sexo.rename(index=situacao_contagio_labels, level='situacao_contagio')
obitos_situacao_contagio_sexo.to_csv(join(default_output,'obitos_situacao_contagio_sexo.csv'))
obitos_situacao_contagio_sexo

In [ ]:
obitos_situacao_contagio_raca_cor = obitos.groupby(['situacao_contagio','raca_cor'])[['id']].count().unstack('raca_cor').fillna(0).astype(int).droplevel(0,1)
obitos_situacao_contagio_raca_cor = obitos_situacao_contagio_raca_cor.rename(index=situacao_contagio_labels, level='situacao_contagio').rename(columns=raca_cor_labels, level='raca_cor')
obitos_situacao_contagio_raca_cor.to_csv(join(default_output,'obitos_situacao_contagio_raca_cor.csv'))
obitos_situacao_contagio_raca_cor

In [ ]:
obitos_mes_situacao_contagio = obitos.groupby(['ano','mes','situacao_contagio'])[['id']].count().unstack('situacao_contagio').fillna(0).astype(int).droplevel(0,1)
obitos_mes_situacao_contagio = obitos_mes_situacao_contagio.rename(columns=situacao_contagio_labels, level='situacao_contagio').rename(index=mes_labels, level='mes')
obitos_mes_situacao_contagio.to_csv(join(default_output,'obitos_mes_situacao_contagio.csv'))
obitos_mes_situacao_contagio

In [ ]:
obitos_mes_situacao_contagio_fator_risco = obitos.groupby(['ano','mes','situacao_contagio','fator_risco'])[['id']].count().unstack(['situacao_contagio','fator_risco']).fillna(0).astype(int).droplevel(0,1).sort_index(axis=1, level=0)
obitos_mes_situacao_contagio_fator_risco = obitos_mes_situacao_contagio_fator_risco.rename(columns=situacao_contagio_labels, level='situacao_contagio').rename(columns=fator_risco_labels, level='fator_risco').rename(index=mes_labels, level='mes')

obitos_mes_situacao_contagio_fator_risco = join_columns_levels(obitos_mes_situacao_contagio_fator_risco)
obitos_mes_situacao_contagio_fator_risco.to_csv(join(default_output,'obitos_mes_situacao_contagio_fator_risco.csv'))
obitos_mes_situacao_contagio_fator_risco

In [ ]:
obitos_mes_situacao_contagio_faixa_etaria = obitos.groupby(['ano','mes','situacao_contagio','fx'])[['id']].count().unstack(['fx','situacao_contagio']).fillna(0).astype(int).droplevel(0,1).sort_index(axis=1, level=0)
obitos_mes_situacao_contagio_faixa_etaria = obitos_mes_situacao_contagio_faixa_etaria.reindex( pd.MultiIndex.from_product([ list(x) for x in  obitos_mes_situacao_contagio_faixa_etaria.columns.levels ], names=obitos_mes_situacao_contagio_faixa_etaria.columns.names),fill_value=0, axis=1)
obitos_mes_situacao_contagio_faixa_etaria = obitos_mes_situacao_contagio_faixa_etaria.rename(columns=situacao_contagio_labels, level='situacao_contagio').rename(index=mes_labels, level='mes')

obitos_mes_situacao_contagio_faixa_etaria = join_columns_levels(obitos_mes_situacao_contagio_faixa_etaria)
obitos_mes_situacao_contagio_faixa_etaria.to_csv(join(default_output,'obitos_mes_situacao_contagio_faixa_etaria.csv'))
obitos_mes_situacao_contagio_faixa_etaria

In [ ]:
nt = Notifica()
fatores = nt.schema.loc[nt.schema['group_name']=='fator_risco','column']
fatores = [ f for f in fatores if f not in ['outras_morbidades','periodo_gestacao','gestante_alto_risco','gestante']]

In [ ]:
obitos_fator_risco = obitos.loc[(obitos['fator_risco']==1)]

obitos_fator_risco_situacao_contagio = {}

for situacao_contagio in sorted(obitos_fator_risco['situacao_contagio'].unique()):
    fator_risco = obitos_fator_risco.loc[obitos_fator_risco['situacao_contagio']==situacao_contagio,fatores]
    n = len(fator_risco)
    fator_risco = fator_risco == 1
    fator_risco = fator_risco.sum(axis=0).to_frame(name='Quantidade')
    fator_risco['%'] = fator_risco['Quantidade'] / n
    # fator_risco['%'] = fator_risco['Quantidade'] / fator_risco['Quantidade'].sum()
    fator_risco = fator_risco.sort_values('%',ascending=False)
    fator_risco.loc['TOTAL DE ÓBITOS','Quantidade'] = n
    obitos_fator_risco_situacao_contagio[situacao_contagio] = fator_risco
    
obitos_fator_risco_situacao_contagio = pd.concat(obitos_fator_risco_situacao_contagio,names=['situacao_contagio','fator_risco'],axis=1)
obitos_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'%')] = (obitos_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'Quantidade')] / obitos_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'Quantidade')].sum()).values
obitos_fator_risco_situacao_contagio = obitos_fator_risco_situacao_contagio.rename(columns=situacao_contagio_labels, level='situacao_contagio')
obitos_fator_risco_situacao_contagio = obitos_fator_risco_situacao_contagio.reorder_levels(['fator_risco','situacao_contagio'], axis=1)

obitos_fator_risco_situacao_contagio = join_columns_levels(obitos_fator_risco_situacao_contagio)

obitos_fator_risco_situacao_contagio.to_csv(join(default_output,'obitos_fator_risco_situacao_contagio.csv'), sep = ';', decimal=',', float_format='%.3f')
obitos_fator_risco_situacao_contagio